In [1]:
!pip install sseclient

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for sseclient: filename=sseclient-0.0.27-py3-none-any.whl size=5570 sha256=82ff0311222840e9ae5e4d7c903a2b578218b38458d48990d9a65fbf3d630a2b
  Stored in directory: /home/aholovenko/.cache/pip/wheels/f9/12/8e/90bd3f4c38f3a0c17681ce222e7762743164876d6dbf04ab71
Successfully built sseclient
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
import random
 

def randomHash(modulus):
    """
    inputs:
    * modulus (tuple)
    output:
    * A random hash function https://docs.google.com/document/d/1Y0O3NAhAHxS9PYcvW4JXIojNzFMXUQEIdDidJ5HNk90/edit
    """
    a, b = random.randint(0,modulus-1), random.randint(0,modulus-1)
    def f(x):
        x = int(str(hash(x))[-1])
        return (a*hash(x) + b) % modulus
    return f

def average(L):
   #Auxiliary function to compute the average value in a list
   return sum(L) / len(L)
 
def numDistinctElements(stream, numParallelHashes=10):
    """
    inputs:
    * stream: list of integers,
    * numParrallelHashes: int
    output:
    * int, the estimated number of different elements on "stream"
    """
    modulus = 2**20 # We use a fixed value for modulus, that will give the size of the hashing functions
    hashes = [randomHash(modulus) for _ in range(numParallelHashes)]
    minima = [modulus] * numParallelHashes
    currentEstimate = 0 #We start  the estimating the different elements as 0

    for i in stream:
        hashValues = [h(i) for h in hashes]
        for i, newValue in enumerate(hashValues):
            if newValue < minima[i]:
                minima[i] = newValue

        currentEstimate = modulus / average(minima) #The estimated value is defined by 1/(x_min+1).

    yield int(currentEstimate)

In [4]:
import json
from sseclient import SSEClient as EventSource

url = 'https://stream.wikimedia.org/v2/stream/recentchange'
wiki = 'enwiki' #Client side filter
counter = 0 
maxEvents = 100 # save n events and stop

numParallelHashes = 10
modulus = 2**20 # We use a fixed value for modulus, that will give the size of the hashing functions
hashes = [randomHash(modulus) for _ in range(numParallelHashes)]
minima = [modulus] * numParallelHashes
currentEstimate = 0 #We start  the estimating the different elements as 0
S = []

for event in EventSource(url):
    if event.event == 'message':
        try:
            change = json.loads(event.data)
        except ValueError:
            continue

        i = hash((change['wiki'], change['user']))
        S.append(i)
        hashValues = [h(i) for h in hashes]
        for i, newValue in enumerate(hashValues):
              if newValue < minima[i]:
                    minima[i] = newValue
  
        currentEstimate = modulus / average(minima) #The estimated value is defined by 1/(x_min+1).
        print(int(currentEstimate), len(set(S)))

        counter += 1 
        if counter > maxEvents:
            break

1 1
2 2
2 3
2 3
2 4
2 4
2 4
4 5
6 6
8 7
8 7
8 8
8 9
10 10
10 10
10 11
10 12
12 13
12 13
12 13
12 14
12 15
12 15
12 16
12 17
12 18
12 18
12 18
12 18
12 19
12 20
12 21
12 21
12 22
12 22
12 23
12 24
12 24
12 25
12 26
12 26
12 27
12 27
12 27
12 27
12 28
12 29
12 30
12 31
12 32
12 33
12 33
12 33
12 33
12 33
12 33
12 33
12 34
12 35
12 35
12 35
12 36
12 36
12 36
12 36
12 36
12 36
12 37
12 38
12 39
12 39
12 40
12 41
12 41
12 41
12 41
12 42
12 42
12 43
12 43
12 43
12 43
12 44
12 44
12 44
12 44
12 44
12 45
12 45
12 45
12 45
12 45
12 45
12 45
12 46
12 46
12 46
12 46
12 47
12 48
12 49
